In [82]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from scipy.stats import wilcoxon
from matplotlib.lines import Line2D

In [68]:
df = pd.read_csv('simulation_results.csv', header=0)

In [69]:
df

simulation  trial   start_time     end_time  duration  duration_seconds  \
0            1      0  02:20:17 PM  02:26:15 PM  00:05:58               358   
1            1      1  02:26:17 PM  02:31:24 PM  00:05:07               307   
2            1      2  02:31:26 PM  02:36:32 PM  00:05:06               306   
3            1      3  02:36:36 PM  02:41:44 PM  00:05:08               308   
4            1      4  02:41:47 PM  02:46:15 PM  00:04:28               268   
..         ...    ...          ...          ...       ...               ...   
95           5      5  01:38:15 PM  01:46:01 PM  00:07:46               466   
96           5      6  01:46:03 PM  01:54:11 PM  00:08:08               488   
97           5      7  01:54:13 PM  02:01:31 PM  00:07:18               438   
98           5      8  02:01:33 PM  02:09:04 PM  00:07:31               451   
99           5      9  02:09:08 PM  02:18:03 PM  00:08:55               535   

        type                                  sequence  
0        opm  bowl, breakfast-cereal, cup, spoon, milk  
1        opm  bowl, breakfast-cereal, cup, spoon, milk  
2        opm  bowl, breakfast-cereal, cup, spoon, milk  
3        opm  bowl, breakfast-cereal, cup, spoon, milk  
4        opm  bowl, breakfast-cereal, cup, spoon, milk  
..       ...                                       ...  
95  baseline  breakfast-cereal, cup, bowl, spoon, milk  
96  baseline  breakfast-cereal, cup, bowl, spoon, milk  
97  baseline  breakfast-cereal, cup, bowl, spoon, milk  
98  baseline  breakfast-cereal, cup, bowl, spoon, milk  
99  baseline  breakfast-cereal, cup, bowl, spoon, milk  

[100 rows x 8 columns]

In [70]:
custom_cmap2 = ["#46327e", "#277f8e", "#4ac16d"]

In [71]:
df[['duration_seconds', 'type']].groupby(['type']).mean()

duration_seconds
type                      
baseline            379.64
opm                 374.72

In [72]:
duration_opm = df.loc[df['type'] == 'opm']['duration_seconds'].mean()

In [73]:
duration_baseline = df.loc[df['type'] == 'baseline']['duration_seconds'].mean()

In [74]:
durations = pd.Series([duration_baseline, duration_opm])

In [75]:
durations.pct_change() * 100

0         NaN
1   -1.295965
dtype: float64

In [76]:
opm = df['duration_seconds'].loc[df['type'] == 'opm'].tolist()
baseline = df['duration_seconds'].loc[df['type'] == 'baseline'].tolist()
len(opm), len(baseline)

(50, 50)

### stat analysis

In [81]:
stat, p = wilcoxon(opm, baseline)
print('Wilcoxon: W = %.3f, p = %.5f' % (stat, p))

Wilcoxon: W = 498.000, p = 0.35593


### plot results

In [11]:
results = pd.DataFrame()

In [84]:
custom_cmap2 = ["#46327e", "#277f8e", "#4ac16d"]
custom_lines2 = [Line2D([0], [0], color=custom_cmap2[1], lw=8, alpha=0.5),
                Line2D([0], [0], color=custom_cmap2[2], lw=8, alpha=0.5)]

In [12]:
results['opm'] = opm
results['baseline'] = baseline
labels = [col for col in results.columns]

In [14]:
%matplotlib qt

ax = sns.violinplot(data=results,
                    palette='viridis',
                    cut=1,
                    inner='quartiles')

plt.setp(ax.collections, alpha=0.7)

ax.set_xticklabels(ax.get_xticklabels(), rotation=0, fontsize=24)

plt.ylabel('duration in seconds', fontsize=24)
#plt.tight_layout(pad=0.9)
plt.show()

In [17]:
simulations_opm = df['simulation'].loc[df['type'] == 'opm'].tolist()
simulations_baseline = df['simulation'].loc[df['type'] == 'baseline'].tolist()

In [18]:
simulations_opm == simulations_baseline

True

In [19]:
results['simulation'] = simulations_baseline

In [20]:
results.groupby('simulation').mean()

opm  baseline
simulation                 
1           324.6     366.2
2           340.5     340.4
3           343.8     329.2
4           434.0     419.4
5           430.7     443.0

In [66]:
labels = [col for col in results.columns]

%matplotlib qt

ax2 = sns.violinplot(data=results.groupby('simulation').mean(),
                    palette=custom_cmap2[1:],
                    cut=2,
                    inner='quartiles',
                    saturation=.8)

plt.setp(ax2.collections, alpha=0.5)

ax2.set_xticklabels(ax.get_xticklabels(), rotation=0, fontsize=24)

plt.ylabel('duration in seconds', fontsize=24)
#plt.tight_layout(pad=0.9)
plt.show()

In [86]:
%matplotlib qt

ax2 = sns.violinplot(x='simulation',
                     y='duration_seconds',
                     hue='type',
                     data=df,
                     palette=custom_cmap2[1:],
                     cut=1,
                     inner='quartiles',
                     saturation=.8)

#ax2.set_xticklabels(ax.get_xticklabels(), rotation=0, fontsize=18)

plt.ylabel('duration in seconds', fontsize=24)
plt.xlabel('simulation', fontsize=24)

plt.setp(ax2.collections, alpha=0.5)
ax2.legend(custom_lines2, ['OPM', 'baseline'], loc='upper left', fontsize=24)

#ax2 = plt.gca()
#leg = ax2.get_legend()
#leg.legendHandles[0].set_color(custom_cmap2[1])
#leg.legendHandles[1].set_color(custom_cmap2[2])

#legend = plt.legend()
#for lh in legend.legendHandles:
#    lh.set_alpha(0.5)

#plt.tight_layout()
#plt.margins(0.01)
plt.show()